<a href="https://colab.research.google.com/github/hrithikpurwar/amazonReviewsSentimentalAnalysis/blob/main/amazonReview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [3]:
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np


In [8]:
import bz2

In [9]:
def fetching(file):
  labels=[]
  texts=[]
  for line in bz2.BZ2File(file):
    x=line.decode("utf-8")
    labels.append(int(x[9])-1)
    texts.append(x[10:].strip())
  return np.array(labels),texts
train_labels, train_texts=fetching("/content/drive/MyDrive/Datasets/train.ft.txt.bz2")
test_labels, test_texts=fetching("/content/drive/MyDrive/Datasets/test.ft.txt.bz2")

In [10]:
train_labels[0]

1

In [11]:
train_texts[0]

'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'

In [12]:
train_labels=train_labels[0:500]
train_texts=train_texts[0:500]

In [13]:
import re
NON_ALPHANUM=re.compile(r'[\W]')
NON_ASCII=re.compile(r'[^a-z0-1\s]')
def normalize(texts):
  normalized_texts=[]
  for text in texts:
    lower=text.lower()
    no_punc=NON_ALPHANUM.sub(r' ',lower)
    no_nonascii=NON_ASCII.sub(r'',no_punc)
    normalized_texts.append(no_nonascii)
  return normalized_texts
train_texts=normalize(train_texts)
test_texts=normalize(test_texts)


In [14]:
train_texts[0]


'stuning even for the non gamer  this sound track was beautiful  it paints the senery in your mind so well i would recomend it even to people who hate vid  game music  i have played the game chrono cross but out of all of the games i have ever played it has the best music  it backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras  it would impress anyone who cares to listen    '

In [15]:
test_texts[0]

'great cd  my lovely pat has one of the great voices of her generation  i have listened to this cd for years and i still love it  when i m in a good mood it makes me feel better  a bad mood just evaporates like sugar in the rain  this cd just oozes life  vocals are jusat stuunning and lyrics just kill  one of life s hidden gems  this is a desert isle cd in my book  why she never made it big is just beyond me  everytime i play this  no matter black  white  young  old  male  female everybody says one thing  who was that singing   '

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(binary=True)
cv.fit(train_texts)
X=cv.transform(train_texts)
X_test=cv.transform(test_texts)

In [17]:
X_test

<400000x5778 sparse matrix of type '<class 'numpy.int64'>'
	with 18647020 stored elements in Compressed Sparse Row format>

In [18]:
X

<500x5778 sparse matrix of type '<class 'numpy.int64'>'
	with 28520 stored elements in Compressed Sparse Row format>

In [20]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val= train_test_split(X, train_labels, train_size=0.7)
for c in [0.01,0.05, 0.25, 0.5, 1]:
  lr=LogisticRegression(C=c)
  lr.fit(X_train, Y_train)
  print(accuracy_score(Y_val, lr.predict(X_val)))

0.7333333333333333
0.7466666666666667
0.7533333333333333
0.76
0.7533333333333333


In [21]:
lr=LogisticRegression(C=0.5)
lr.fit(X_train, Y_train)
print(accuracy_score(Y_val, lr.predict(X_val)))

0.76


In [22]:
import pickle
pickle_out = open("lr.pkl","wb")
pickle.dump(lr, pickle_out)
pickle_out.close()

In [23]:
import pickle
pickle_out = open("lr.pkl","wb")
pickle.dump(lr, pickle_out)
pickle_out.close()

In [24]:
import pickle
pickle_out = open("cv.pkl","wb")
pickle.dump(cv, pickle_out)
pickle_out.close()

In [44]:
lr.predict(cv.transform(['you are very bad']))

array([0])

In [32]:
lr.predict(cv.transform(['you are very good']))

array([1])

In [48]:
lr.predict(cv.transform(['didnt run off of USB bus power: Was hoping that this drive would run off of bus power, but it required the adapter to actually work. :( I sent it back.']))

array([0])